# 準備
## ライブラリのインストール
Colaboratory環境で実行する場合は`japanaize_matplotlib`をインストールする必要があります。

SageMaker環境で以下を実行する必要はありませんが、実行しても害はありません。

In [1]:
%pip install japanize_matplotlib

Note: you may need to restart the kernel to use updated packages.


## ライブラリのインポート

この資料で利用するライブラリを一括してインポートします。

SageMakerやColaboratoryで接続がタイムアウトした時は、
以下のセルを再度実行して下さい。

In [ ]:
#ライブラリーのインポート
import sklearn
import numpy as np
from sklearn import manifold
from sklearn import model_selection
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import japanize_matplotlib
import math
import random
import pandas as pd

# 2章 距離関数による機械学習
本章では1.4節で述べた「類似度に基づいて学習を行う機械学習のフレームワーク」に従い、著者当てゲームを実装していきます。

機械学習ではモデルが正しい予測をすることが最も重要です。一方で、顕著な成果をあげている深層学習でモデルを解釈することができないように、モデルは人間にとって理解しやすいとは限りません。

<font color= "red">**機械学習は探索の科学**</font>であり、探索の方法論を研究する学問領域です。仮説の検証が目的の統計学とは異なり、モデルの探索に重きを置いています。

その探索対象は学習アルゴリズムだけではなく、データの加工方法や類似度関数など多岐にわたります。本章では以下で紹介する探索サイクルに沿って正確性の高いモデルを見つける過程で、機械学習についての基礎を身に着けることを目的としています。

### 探索対象

1. 特徴
1. データの表現
1. 類似度関数
1. 学習アルゴリズムと最適ハイパーパラメータ
1. モデル

### 探索のサイクル

以下のステップのサイクルを繰り返し、正確性の高い予測のためのモデルを探索します。

1. 特徴の設計
1. データの加工
1. 類似度関数の設計
1. 学習アルゴリズムの選択
1. ハイパーパラメータの最適化
1. モデルの評価

## 著者当てゲーム
短編小説のテキストからその著者を当てるゲームを通して、 機械学習における探索のサイクルを体験しましょう。その過程で、機械学習で用いられる基本的な概念などについても学べるはずです。

### 著者当てゲームで使うデータについて

今回の著者当てゲームでは[青空文庫](https://www.aozora.gr.jp)からテキストデータをダウンロードして利用します。
> 青空文庫には15万件を超える作品が収録されているが、
そのいずれも、著作権が消滅しているか、著作者から許諾が得られている作品である。

また、今回は明治時代の文豪である
芥川龍之介（1892年 - 1927年）と菊池寛（1888年 - 1948年)を取り上げます。
二人の作品の著作権は消滅しています。
> 二人は、同時代の語彙を共有していて、また、共に短編作品が多いことから、
同じ条件で比較がしやすい。
> 蛇足であるが、菊池寛は芥川賞設立の発起人のひとりである。

芥川龍之介の作品からは、以下の20編を学習用のデータとして利用します。

> 羅生門、妖婆、藪の中、貉、鼻、歯車、トロッコ、杜子春、俊寛、侏儒の言葉、邪宗門、将軍、死後、アグニの神、或る日の大石内蔵助、おぎん、河童、煙管、蜘蛛の糸

菊池寛についても、同数の20編を選びました。

> 姉川の戦い、ある恋の話、入れ札、M公爵と写真師、屋上の狂人、恩讐の彼方に、女強盗、恩を返す話、義民甚平、勲章を貰う話、極楽、出世、勝負事、大力の物語、藤十郎の恋、ある抗議書、身投げ救助業、無名作家の日記、若杉裁判長、奉行と人相学

<!-- 
dataフォルダに青空文庫からダウンロードした上記40編の短編小説のテキストファイルを保存してあります。

(＊dataフォルダの導入がうまくいかない場合は、以下のデータダウンロードのセルを実行することで、この先のコードも問題なく実行することができます。)
-->



### テキストデータの確認
青空文庫からダウンロードしたデータは、dataフォルダにテキストファイルとして保存されています。

data/rashomon.txtは「羅生門」の全文のテキストファイルです。
data/rashomon.txtの全文を表示してみます。

In [ ]:
open("data/rashomon.txt", "r", encoding = "shift-jis").read()

同様にdata/ahen.txtの全文を表示してみます。

In [ ]:
open("data/ahen.txt", "r", encoding = "shift-jis").read()

## 2.2 機械学習における探索のサイクル
正確性の高いモデルを見つけるために以下のステップを繰り返します。ここでは簡単、探索のサイクルについて紹介します。

### 2.2.1 特徴設計
データを分析する目的に照らして、注目する特徴を定めるステップです。


### 2.2.2 データの加工
データから特徴を抽出し、機械学習の処理に適した形式に整形するステップです。
また、特徴の欠損値や外れ値などを処理したり、特徴値のスケールをそろえるための正規化処理なども本ステップで行います。


### 2.2.3 類似度関数の設計
データの類似度を定量的に計算する類似度関数を決めるステップです。類似度関数を決めることは、類似性を定義づけることになるので、非常に重要なステップとなります。

### 2.2.4 学習アルゴリズムの選択とハイパーパラメータ最適化
データ分析の目的や選択した類似度関数に照らして、学習アルゴリズムを定めます。
また、学習アルゴリズムは、事前に人間が定めておくべきハイパーパラメータを含むことがあります。
最適なハイパーパラメータの値もこのステップで探索します。

### 2.2.5 モデル評価
訓練データを学習アルゴリズムに入力すると、予測のためのモデルを得ることができます。作成したモデルに対してテストデータを入力し、予測値を計算し、答え合わせを行うことでモデルの性能を評価するステップです。

### 2.2.6 スパイラルアップ
以上のステップを性能の良いモデルが得られるまで繰り返し探索を行います。

## 2.3 特徴設計
ここでは「著者判定を行うために、与えられたテキストのどんな特徴に注目すべきか」を検討します。

今回取り扱う生データはテキストであり、1章の例で使用した「ワイン価格のデータセット」のような「特徴を整理した表形式」のデータではありません。

コンピュータにとってテキストは文字の羅列であり、それ以上の意味を持ちません。
そのため、私たちが「著者の文体の違いを表現できるような特徴」を設計し テキストから抽出する必要があります。今回は以下の仮設に基づき、特徴を定めます。

---
<font color= "red">**文体は自立語である名詞・形容詞・動詞・副詞の分布によって特徴づけられる**</font>


---

## 2.4 データの加工
ここでは上記の仮説に従って、テキストデータから**特徴**を抽出します。

具体的には、テキストに現れる自立語（名詞・形容詞・動詞・副詞）の全単語について出現回数を求め、
表形式にまとめます。
一つの作品について、作品中に現れる各自立語の個数が、この作品の特徴となります。

実は40作品には16,307個の自立語が現れますので、
各作品は、それぞれの自立語の出現回数である16,307個の数字の並び（**ベクトル**）によって特徴付けられます。

実際に以下の手順に従って特徴抽出を行いましょう。

1. 原文テキストの抽出\
テキストデータから余分な要素を取り除き**原文テキストを復元**します。
1. 形態素解析\
作品毎に形態素解析を行うことでテキストを単語に分解します。
2. 名詞・形容詞・動詞・副詞の抽出
3. 名詞・形容詞・動詞・副詞の数え上げ

---
### 1.原文テキストの抽出
まずステップ1では、原文テキストの抽出を行います。


青空文庫からダウンロードしたテキストファイルには、原文テキストにはない要素が含まれています。

```
\u3000 ある日の暮方の事である。一人の下人《げにん》が、 羅生門《らしょうもん》の下で雨やみを待っていた。\n\u3000 広い門の下には、この男のほかに誰もいない。ただ、所々|丹塗《にぬり》の剥《は》げた、大きな円柱《まるばしら》に、 蟋蟀《きりぎりす》が一匹とまっている。
...(中略)... 
  下人はとうとう、老婆の腕をつかんで、無理にそこへ※［＃「てへん＋丑」、第4水準2-12-93］《ね》じ倒した。丁度、鶏《にわとり》の脚のような、骨と皮ばかりの腕である。\n「何をしていた。云え。云わぬと、これだぞよ。」
  \n\u3000下人は、老婆をつき放すと、いきなり、太刀の鞘《さや》を払って、白い鋼《はがね》の色をその眼の前へつきつけた。
...(中略)... 
そうし て、そこから、短い白髪《しらが》を倒《さかさま》にして、 門の下を覗きこんだ。外には、ただ、黒洞々《こくとうとう》 たる夜があるばかりである。\n\u3000 下人の行方《ゆくえ》 は、誰も知らない。
```

>- ユニコードエスケープは、UTF-8やUTF-16で符号化された文字を 直接指定する方法で、例えば、\u3000 は全角のスペースを表します。
>- \nは改行を表す制御文字です。
>- 《げにん》などの漢字の読みの表示は、青空文庫で決めている書式です。
>- JIS水準が高い漢字は、注釈記号を使って表示されます。
例えば、※\[#「てへん+丑」、第 4 水準 2-12-93\]などです。
>- 注釈や漢文の返り点なども注釈記号を用いて記述されます。注釈記号は青空文庫で決めている書式です。

これらの要素は形態素解析の結果に影響を及ぼす可能性のあるものがあります。

>- ユニコードエスケープと制御記号は標準で定められた書式で、形態素解析に影響を与えないので、ここでは除去しません。
>- ルビや注釈は青空文庫固有の書式で記述されているため形態素解析を実行する上で障害となる可能性があります。JIS水準が高い漢字を含む語は読みで置き換え、その他の注釈とルビは削除します。

#### 例1

> 下人はとうとう、老婆の腕をつかんで、無理にそこへ※［＃「てへん＋丑」、第4水準2-12-93］《ね》じ倒した。丁度、鶏《にわとり》の脚のような、骨と皮ばかりの腕である。
「何をしていた。云え。云わぬと、これだぞよ。」
　下人は、老婆をつき放すと、いきなり、太刀の鞘《さや》を払って、白い鋼《はがね》の色をその眼の前へつきつけた。

> 下人はとうとう、老婆の腕をつかんで、無理にそこへねじ倒した。丁度、鶏の脚のような、骨と皮ばかりの腕である。
「何をしていた。云え。云わぬと、これだぞよ。」
　下人は、老婆をつき放すと、いきなり、太刀の鞘を払って、白い鋼の色をその眼の前へつきつけた。
 
#### 例2

> 僕はこの頃漫然と兪※［＃「木＋越」、第3水準1-86-11］《ゆゑつ》の「右台仙館筆記《うたいせんくわんひつき》」を読んでゐるうちにかう云ふ俗伝は支那人の中にもあつたと云ふことを発見した。それは同書の中に掲げた「賈慎庵《かしんあん》」の話に出合つたからである。
　賈慎庵は何でも乾隆《けんりゆう》の末の老諸生の一人だつたと云ふことである。それが或夜の夢の中に大きい役所らしい家の前へ行つた。家は重門｜尽《ことごと》く掩《おほ》ひ、闃《げき》としてどこにも人かげは見えない。「正に徘［＃「徘」は底本では「俳」］徊《はいくわい》の間、俄《には》かに数人あり、一婦を擁して遠きより来り、この門の外に至る。
 
> 僕はこの頃漫然とゆゑつの「右台仙館筆記」を読んでゐるうちにかう云ふ俗伝は支那人の中にもあつたと云ふことを発見した。それは同書の中に掲げた「賈慎庵」の話に出合つたからである。
　賈慎庵は何でも乾隆の末の老諸生の一人だつたと云ふことである。それが或夜の夢の中に大きい役所らしい家の前へ行つた。家は重門｜尽く掩ひ、闃としてどこにも人かげは見えない。「正に徘徊の間、俄かに数人あり、一婦を擁して遠きより来り、この門の外に至る。

### 2.形態素解析
次にステップ2では、形態素解析を行います。

- 文字列を形態素（単語）に分解
- 各形態素の品詞を特定
- 活用形を標準形（終止形）に変換

> 文中で意味をもつ文字の並びの最小単位を、自然言語処理では**形態素**と呼ぶ。
単純のため単語と同義と考えてもよいが、
複数の単語が結合して別の単語を構成することがある一方、
形態素はそれ以上分解できない最小の単位を表す。
テキストの文法構造を明らかにするためには、
まず、単なる記号の列であるテキストを、形態素の集まりに整理し、
それぞれの形態素に対して、その品詞と用法を特定することが必要である。
このような処理のことを、**形態素解析**と呼ぶ。


MeCabという形態素解析のライブラリーを利用した時の形態素解析の例を示します。

**原文**
> 一人の下人が、羅生門の下で雨やみを待っていた。

**形態素解析の結果**
> *:BOS/EOS:*   一:名詞:数   人:名詞:接尾   の:助詞:連体化   下人:名詞:一般   が:助詞:格助詞   、:記号:読点   羅生門:名詞:固有名詞   の:助詞:連体化   下:名詞:一般   で:助詞:格助詞   雨:名詞:一般   やみ:名詞:一般   を:助詞:格助詞   待つ:動詞:自立   て:助詞:接続助詞   いる:動詞:非自立   た:助動詞:*   。:記号:句点   *:BOS/EOS:*

### 3. 自立語（形容詞・動詞・名詞・副詞）の抽出

ステップ3では、形態素解析の結果から、形容詞、動詞、名詞、副詞のみを残して、
残りを除去します。

**原文**
> 一人の下人が、羅生門の下で雨やみを待っていた。

**自立語抽出の結果**
> '一:名詞:数', '人:名詞:接尾', '下人:名詞:一般', '羅生門:名詞:固有名詞', '下:名詞:一般', '雨:名詞:一般', 'やみ:名詞:一般', '待つ:動詞:自立', 'いる:動詞:非自立']
一:名詞:数   人:名詞:接尾   下人:名詞:一般   羅生門:名詞:固有名詞   下:名詞:一般   雨:名詞:一般   やみ:名詞:一般   待つ:動詞:自立   いる:動詞:非自立

### 4. 自立語（形容詞・動詞・名詞・副詞）の数え上げ

ステップ3までで、任意のテキストのルビと注釈を削除し、全ての形態素を含むリストを作成しました。
ステップ4では、各形態素がテキスト中に何回出現するかを数え上げます。

**原文**
> 私の兄は私より背が高いです。あなたの弟は私よりも背が高いですか?

**自立語抽出の結果**
> 私:名詞:代名詞   兄:名詞:一般   私:名詞:代名詞   背:名詞:一般   高い:形容詞:自立   あなた:名詞:代名詞   弟:名詞:一般   私:名詞:代名詞   背:名詞:一般   高い:形容詞:自立

**自立語数え上げの結果**
> '私:名詞:代名詞': 3, '背:名詞:一般': 2, '高い:形容詞:自立': 2, '兄:名詞:一般': 1, 'あなた:名詞:代名詞': 1, '弟:名詞:一般': 1

### 40作品に対して特徴を抽出

以上の4つのステップに従い、
ダウンロードした芥川龍之介・菊池寛の各作品から特徴を抽出した結果を保存したファイルを読み込みます。

In [ ]:
akutagawa_name = np.load('data/akutagawa_name.npy', allow_pickle=True)
kikuchi_name = np.load('data/kikuchi_name.npy', allow_pickle=True)
hgram = np.load('data/hgram.npy', allow_pickle=True)
words = np.load('data/words.npy', allow_pickle=True)
print('芥川龍之介の作品は以下の20編')
print(' '.join(akutagawa_name))
print('\n菊池寛の作品は以下の20編')
print(' '.join(kikuchi_name))
print('\n40編の作品に現れる語の総数は{}語'.format(len(words)))

各作品の特徴ベクトルの一部を表示してみます。

In [ ]:
# (表形式のデータの作成)
columns = ['する:動詞:自立', 'いる:動詞:非自立', 'の:名詞:非自立', 
           '事:名詞:非自立',
           '云う:動詞:自立',
           '老婆:名詞:一般',
           'よう:名詞:非自立',
           'なる:動詞:自立']

def count(n, w):
    if w in hgram[n]:
        return hgram[n][w]
    else:
        return 0

body = []
for n in range(20):
    temp = [count(n, w) for w in columns]
    temp.append('芥川')
    body.append(temp)
for n in range(20):
    temp = [count(n+20, w) for w in columns]
    temp.append('菊池')
    body.append(temp)

df = pd.DataFrame(body)
columns.append('著者')
df.columns = columns
df.index = np.hstack([akutagawa_name, kikuchi_name])
df

各作品に対して、8個の数値が与えられています。
実際には、各作品の特徴は16,307個の数値の並びですので、
上の表は作品の特徴のごくごく一部を表示しているにすぎません。

## 2.5 類似度関数の設計
まず、作品の特徴の分布、即ち、作品中に現れる名詞・形容詞・動詞・副詞の出現頻度の分布を作品ごとに表示し、観察します。
その上で、私たち人間が特徴の分布を眺めていても、芥川龍之介の作品と菊池寛の作品を分類するための規則やパターンを見出すことが難しいことを体感してみます。

始めに、「羅生門(芥川)」と「姉川の戦い(菊池)」について、2.4節で得られた形態素の分布を比較してみましょう。(試しに以下の8項目の特徴で比較)
1. 'する:動詞:自立'
1. 'いる:動詞:非自立'
1. 'の:名詞:非自立'
1. '事:名詞:非自立'
1. '云う:動詞:自立'
1. '老婆:名詞:一般'
1. 'よう:名詞:非自立'
1. 'なる:動詞:自立'

In [ ]:
pip install japanize-matplotlib # 日本語版matplotlibのインストール

In [ ]:
# 前記の8項目の特徴に関して、「羅生門」と「姉川の戦い」の分布を比較
import matplotlib.pyplot as plt
import japanize_matplotlib
df_wo_author = df[df.columns[:-1]]

_, axes = plt.subplots(1, 2, figsize=[15,6])
fig1 = df_wo_author.loc['羅生門'].plot.bar(title='羅生門（芥川龍之介）', ax=axes[0], fontsize=16)
fig1.axes.title.set_size(24)
fig2 = df_wo_author.loc['姉川の戦い'].plot.bar(title='姉川の戦い（菊池寛）', ax=axes[1], fontsize=16)
fig2.axes.title.set_size(24)

axes[0].set_ylim([0,160])
axes[1].set_ylim([0,160])
plt.show()

同じ8項目の特徴に関して、全作品の分布を比較してみましょう。

In [ ]:
 # 8項目の特徴の分布を全作品で比較
_, axes = plt.subplots(10, 4, figsize=[20,30])
# _, axes = plt.subplots(8, 5, figsize=[20,25])
for n in range(40):
    title = df.index[n]
    df_wo_author.loc[title].plot.bar(title=title+('（芥川）' if n < 20 else '（菊池）'), 
                                     ax=axes[int(n/4), n%4],
                                     xticks=[]
                                    )
    axes[int(n/4), n%4].set_ylim([0, 400])
    axes[int(n/4), n%4].xaxis.set_visible(False)
plt.show()

作品ごとに大きくばらついていることが分かると思います。
この分布を眺めて、芥川の作品に共通に見られるパターン、
菊池の作品に共通に見られるパターンで、
芥川の作品と菊池の作品を区別できるものを見つけることはできるでしょうか？

8個の特徴だけでは不十分なので、全16301項目の特徴に関して、同じように比較をしましょう。
まず、「羅生門(芥川)」と「姉川の戦い(菊池)」の分布を表示します。

In [ ]:
 # 「羅生門(芥川)」と「姉川の戦い(菊池)」の分布の比較
import numpy as np

_, axes = plt.subplots(1, 2, figsize=[15,5])

y = [count(0, w) for w in words]
axes[0].plot(range(len(words)), y)
axes[0].set_title('羅生門（芥川龍之介）', fontsize=24)
axes[0].set_ylim([0, 160])
axes[0].xaxis.set_visible(False)
axes[0].set_yticklabels(np.arange(0, 170, 20), fontsize=16)

y = [count(20, w) for w in words]
axes[1].plot(range(len(words)), y)
axes[1].set_title('姉川の戦い（菊池寛）', fontsize=24)
axes[1].set_ylim([0, 160])
axes[1].xaxis.set_visible(False)
axes[1].set_yticklabels(np.arange(0, 170, 20), fontsize=16)
plt.show()

更に、全16301項目の特徴に関して、全作品の分布を比較してみましょう。

In [ ]:
 # 全作品の全単語の分布の比較(2分くらい実行時間がかかります)
_, axes = plt.subplots(10, 4, figsize=[20,30])
for n in range(40):
    title = df.index[n]
    y = [count(n, w) for w in words]
    ax = axes[int(n/4), n%4]
    ax.plot(range(len(words)), y)
    ax.set_ylim([0, 400])
    ax.set_title(title+('（芥川）' if n < 20 else '（菊池）'), fontsize=20)
    ax.xaxis.set_visible(False)
plt.show()

人の目で上図を観察して、芥川龍之介の作品と菊池寛の作品を分類するための規則やパターンを見出すことは難しいことがわかると思います。

分類の目標は同じ著者の作品間の分布の違いは受容し、違う著者の作品間の分布の違いを検出することです。私達はこの複雑な問題を解くために、適切な類似度関数を設計する必要があります。

> 類似度関数は2変数実関数$\sigma(x, y)$であり、厳密には以下のように定義されています。
>
> | 関数 | 定義 |
> |:----|:----|
> |**類似度関数** | $x$と$y$が似ているほど、大きな値を返す |
> |**非類似度関数** | $x$と$y$が似ているほど、小さな値を返す |
>
> 便宜上、これらを区別せず類似度関数と呼ぶこととします。

#### ユークリッド距離（$L^2$ノルム）
$L^2$ノルムは、多変量解析・統計で基本とするベクトルの長さの定義です。

データは16301次元のベクトルとして表現されるので、ユークリッド距離を(非)類似度関数として利用してみることとします。

一つのデータを16301次元のベクトル空間の点と考えて、二つのデータがどの程度似ているかは二点間の距離を指標にして考えます。二点間の距離は二点を結ぶベクトルの長さで定義します。

$$
d_{L^2}\left((v_1, \dots, v_{16301}), (w_1, \dots, w_{16301})\right) 
= \sqrt{\sum_{i=1}^{16301} (v_i - w_i)^2} 
$$

試しに、芥川龍之介の「羅生門」と菊池寛の「姉川の戦い」のユークリッド距離を計算してみましょう。

In [ ]:
# 「羅生門」と「姉川の戦い」のユークリッド距離
def dl2(dictx, dicty):
    sq = 0
    for key in set(dictx.keys()) | set(dicty.keys()):
        x, y = 0, 0
        if key in dictx: 
            x = dictx[key]
        if key in dicty:
            y = dicty[key]
        sq += (x - y)**2
    return math.sqrt(sq)
        
print('「羅生門」と「姉川の戦い」のユークリッド距離 = ', dl2(hgram[0], hgram[20]))

#### 距離行列の計算
全てのデータ（作品）間の距離を計算し、$40 \times 40$の行列にまとめたものを、
**距離行列**と呼びます。

> $i$番目と$j$番目のデータの距離が、距離行列の$i$行・$j$列の値で与えられます。

距離行列は類似度関数として距離関数を使用した場合の類似度行列です。
- 類似度行列は全ての訓練データのペアの間の類似度を含んでいます。
- 類似度行列は**対称行列**です。即ち、左上から右下への対角線を軸として、値は対象になります。
- 類似度関数として距離関数を用いた場合、類似度行列の対角成分は全て0となります。

以下のセルでは、$40\times 40$の距離行列を計算した後、
先頭の5行・5列のみを表示します。

In [ ]:
 # 距離行列の作成
temp = [] 
for dictx in hgram:
    temp.append(list(map(lambda dicty: dl2(dictx, dicty), hgram)))
dl2_matrix = np.array(temp) # 距離の行列

print(dl2_matrix[:5, :5])

## 2.6 学習アルゴリズムの選択
ここでは、著者判定の学習アルゴリズムとして、
距離に基づく分類アルゴリズムとしては非常に一般的な
**k-NNアルゴリズム**を指定します。

k-NNアルゴリズムについて理解したのち、
k-NNアルゴリズムの有効性を(参考程度に)事前に確認する手段として**次元圧縮によるデータの可視化**を行います。


### 2.6.1 k-NN アルゴリズム
---

**テストデータに最も距離が近い$k$個の訓練データを探索し、これらの訓練データのラ
ベルの多数決により、ラベルを予測する**

---

> k はハイパーパラメータと呼ばれ、k-NN アルゴリズムを訓練データに 適用する前に決定しておく必要があります。k の値によって、k-NN アルゴリズムの結果が異なってくるので、最適な kの値を選ぶことは重要です。
>
> 下の例では、k = 3 ではテストデータのクラスは「芥川」、k = 5 ではテストデータのクラスは「菊池」と分類されます。最適な k を決定する方法については、次の節で触れます。

まずはk-NNアルゴリズムについて、直感的に理解していきましょう。ひとまず、次のセルを実行してください。

In [ ]:
 # これはk-NNアルゴリズムを説明するための図(これまでに作成した40作品のデータとは全く関係ないことに注意)
plt.rcParams["figure.figsize"] = (5, 5)
plt.axis('off')

plt.scatter([-1,.5,.6,.8,2], [2,-.5,.3,1.8,-1], c='black', 
            marker='+', s=100, label='芥川')
plt.scatter([-2,-1,0,-.8,1], [-2,0,1.5,-.8,-1.6], c='black', 
            marker='o', label='菊池')
plt.scatter([0], [0], c='black', marker='s', label='テスト')
plt.plot([np.cos(t) for t in np.linspace(0, 2*np.pi, 50)],
         [np.sin(t) for t in np.linspace(0, 2*np.pi, 50)],
         label='$k = 3$', c='black'
        )
plt.plot([1.5*np.cos(t) for t in np.linspace(0, 2*np.pi, 50)],
         [1.5*np.sin(t) for t in np.linspace(0, 2*np.pi, 50)],
         label='$k = 5$', c='black', linestyle='dashed'
        )
plt.grid()
plt.legend(fontsize=14, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.show()

■はテストデータで、●と**＋**は訓練データで、それぞれ菊池と芥川の作品を表しています。
- $k=3$ならば、テストデータから距離が小さい3個の訓練データを見つけます(実線円)。●が一つ、**＋**が二つなので、多数決によって■を芥川の作品と予測します。
- $k=5$ならば、テストデータから距離が小さい5個の訓練データを見つけます(点線円)。今回は●が三つ、**＋**が二つなので、多数決によって■を菊池の作品と予測します。


### 2.6.2 k-NN アルゴリズムの適性

k-NNのアルゴリズの適性を以下の例を用いて解説します。
*   分布が分離していてk-NNが適している例
*   分布が重なっていてk-NNが適さない例
*   これらの中間の例



In [ ]:
 # 基準の分布
plt.rcParams["figure.figsize"] = (10, 5)
n = 100
x1 = np.random.randn(n)
y1 = np.random.randn(n)
dx2 = np.random.randn(n)
y2 = np.random.randn(n)

In [ ]:
 # 分布が分離していてk-NNが適している例
plt.rcParams["figure.figsize"] = (10, 5)
a = 10
plt.scatter(x1, y1, color='black', marker='+', s=100)
x2 = np.ones(n)*a + dx2
plt.scatter(x2, y2, color='black', marker='o')
plt.axis("off")
plt.show()

分布が分離しているケースでは、k-NNは1に近い確率で予測を的中させることができます。

In [ ]:
 # 分布が重なっていてk-NNが適さない例
plt.rcParams["figure.figsize"] = (5, 5)
a = 0
plt.scatter(x1, y1, color='black', marker='+', s=100)
x2 = np.ones(n)*a + dx2
plt.scatter(x2, y2, color='black', marker='o', alpha=0.5)
plt.axis("off")
plt.show()

このように、分布が同一であるケースでは、k-NNは原理的に1/2の確率でしか予測を的中させることができません。

In [ ]:
 # 中間の例
plt.rcParams["figure.figsize"] = (7, 5)
a = 3
plt.scatter(x1, y1, color='black', marker='+', s=100 )
x2 = np.ones(n)*a + dx2
plt.scatter(x2, y2, color='black', marker='o', alpha=0.5)
plt.axis("off")
plt.show()

二つの分布が交差する領域では、k-NNの予測の的中率は1/2に近いと想像できますので、全体の的中率は的中率は1/2と1の間になる筈です。

### 2.6.3 次元圧縮による可視化
k-NNアルゴリズムの適性を(参考程度に)見積もるために、著者判定のための特徴ベクトルに関しても2.6.2節で行ったようなデータの分布を確認したいところですが、ベクトルの次元が16301次元であるため、可視化することができません。

そこで、**次元削減**と呼ばれる手法を用いて、16301次元を3次元に圧縮して可視化してみましょう。

先に計算した作品間の距離を「できるだけ」保存するように、三次元空間にプロットします。
まず、40編の作品のそれぞれに対して、
$x$座標、$y$座標、$z$座標の3つの値を計算します。

In [ ]:
# MDSで16301次元のベクトル達をを3次元に無理やり圧縮
mds = manifold.MDS(n_components=3, dissimilarity="precomputed", random_state=6)
pos = mds.fit_transform(dl2_matrix)
pos

MDSで三次元に投影した結果を可視化してみます。

In [ ]:
 # MDSで3次元に圧縮
l1 = len(akutagawa_name)
l2 = len(kikuchi_name)

fig = go.Figure(
    layout=go.Layout(
    title="ユークリッド距離に基づくデータの分布（MDSによる次元圧縮）" ,
    showlegend=True,
    legend=dict(x=0.7, y=0.99, xanchor='left', yanchor='top', font=dict(size=16))
    )
)

fig.add_trace(go.Scatter3d(
        x = pos[0: l1,0], y = pos[0: l1,1], z = pos[0: l1,2],
        mode = 'markers',
        marker = dict(symbol='cross', color='black', size=5, 
                      line=dict(width=0), opacity=1 ),
        name = '芥川'
        )
)

fig.add_trace(go.Scatter3d(
        x = pos[l1: l1+l2, 0], y = pos[l1: l1+l2, 1], z = pos[l1: l1+l2, 2],
        mode = 'markers',
        marker = dict(symbol='circle', color='black', size=5, 
                      line=dict(width=0), opacity=1),
        name = '菊池'
        )
)



グラフをドラッグして、希望の方向から分布を観察することができます。

芥川の作品のプロットと菊池の作品のプロットは分かれているように見えますが、分布は隣接しているため、k-NNで予測を行った場合の正解率は高くならない懸念もあります。

## 2.7 ハイパーパラメータの最適化
ここでは、学習アルゴリズムとして採用したk-NNアルゴリズムについて、ハイパーパラメータである$k$の最適な値をグリッド探索によって決定します。\
その過程で、限りあるデータを上手く活用することで予測の正確性と正解率の信頼性を高める「交差検証」と、モデルを評価する上で重要な役割を果たす「混同行列」についての理解を深めます。

**$k$を決定する方針**
1. $k$の値を1から10まで変えて学習と予測を実行

1. 各$k$の値に対して予測の正解率（正解予測数の全予測数に対する比率）を計算

1. 最も高い正解率を出した$k$を最適値として採用

最適なハイパーパラメータを決定するに当たり、
一定の範囲において総当たりで予測性能を評価し、
最も良好な予測性能を示した値を最適値とする探索手法を**グリッド探索**と呼びます。


40個のデータを**訓練データ**として利用して学習を実行し、
同じ40個のデータを**テストデータ**として予測を行ってみましょう。

In [ ]:
# ハイパーパラメータの値と正解率の関係を得る
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter("ignore", FutureWarning)

l1 = len(akutagawa_name)
l2 = len(kikuchi_name)
labels = [0]*l1 + [1]*l2

y = []
for k in range(1, 11):
    clf = KNeighborsClassifier(metric='precomputed', n_neighbors=k)
    clf.fit(dl2_matrix, labels)
    pred = clf.predict(dl2_matrix)
    y.append(accuracy_score(labels, pred))

# ハイパーパラメータの値と正解率の関係の可視化
plt.ylim(0, 1)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('$k$', fontsize=16)
plt.ylabel('正解率', fontsize=16)
plt.bar(range(1, 11), y)
plt.grid()
plt.show()

正解率は、$k = 1$の時に1.0、$k = 3$の時に0.9となり、
一見うまくいっているように見えます。

しかし、この一見よい結果は、**カンニング効果**、つまり、
試験問題の答えを予め知っていたことによるものです。

つまり、正確な予測性能の評価を行うためには、
**訓練データと異なるテストデータで評価しなければならない**
という原則があるのですが、
この原則に従わなかったことから不適切に高い正解率が得られた訳です。

利用できるデータ数に限りがあるとき、次のような問題が起こります。

- 訓練データが少ないと、学習が十分に行われず、予測性能が低くなる
- テストデータが少ないと、偶然により予測の分布が偏る可能性があり、評価の信頼性が低くなる

例えば、40個あるデータを半分に割って、
学習用に20個、テスト用に20個使うものとすると、
訓練データ数・テストデータ数のどちらも不十分であり、
適切な評価を行えない可能性が高いと考えられます。

利用できるデータ数が少ないときには、**交差検証法**が有効です。
交差検証法の詳細については、別の資料で説明しますが、
以下では交差検証法により適切に計算された正解率を示します。

In [ ]:
 # ハイパーパラメータの値と正解率の関係を得る
parameters = [{'metric': ['precomputed'], 
               'n_neighbors': list(range(1,11))}]
clf = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)
clf.fit(dl2_matrix, labels)

x = []
y = []

params = clf.cv_results_['params']
mean_test_score = clf.cv_results_['mean_test_score']
std_test_score = clf.cv_results_['std_test_score']
for p, m, s in zip(params, mean_test_score, std_test_score):
    print(f"{m:.3f} (+/- {s/2:.3f}) for {p}")
    x.append(p['n_neighbors'])
    y.append(m)

 # ハイパーパラメータの値と正解率の関係の可視化
plt.ylim(0, 1)
plt.yticks(np.arange(0, 1.1, 0.1))
plt.xlabel('$k$', fontsize=16)
plt.ylabel('正解率', fontsize=16)
plt.bar(x, y)
plt.grid()
plt.show()

正解率の一番高い$k$をハイパーパラメータの最適値とします。

In [ ]:
 # kの最適値
clf.best_estimator_.n_neighbors

## 2.8 モデルの評価
正解率の最大値を求めます。

In [ ]:
 # 正解率の最大値
scores = model_selection.cross_val_score(clf.best_estimator_, X = dl2_matrix, y = labels, cv = 5)
np.average(scores)

正解率が低く、さらに探索をする必要があります。